<a href="https://colab.research.google.com/github/RafaelKampa/ConstrucaoInterpretadores/blob/main/CosSImilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>TRABALHO RECUPERAÇÃO DE TEXTOS – TF-IDF</b></h1>
<h2>Alunos: Bernardo Zeni Diniz, Lucas Camargo Vianna, Pedro Leonardo Alves da Silva, Rafael Gilberto Kampa</h2>


<h1>ENUNCIADO</h1>
<br>
Sua tarefa será gerar uma matriz de cossenos dos ângulos entre todos os vetores criados na matriz termo documento, criada com o algoritmo Tf-Idf. As colunas desta matriz serão números que indiquem o documento que criou o vetor (sentença original). Da mesma forma, as linhas desta matriz serão identificadas por números que identifiquem o documento que criou o vetor (senteça) original. Desta forma teremos uma matriz quadrada cujo lado é igual ao número de vetores que você criou e cuja diagonal será zero.


In [44]:
import requests
import re
import numpy as np

# # Salva o nome do arquivo em uma variável global para futuros usos
nome_arquivo_gerado = "BagOfWords.txt"

# Lê o arquivo de texto do URL fornecido
response = requests.get("https://www.gutenberg.org/files/53106/53106-8.txt")
texto = response.text

# Separa o texto em documentos usando "." "," ";" como separadores e substitui "\n" e "\r" por um espaço em branco
separadores = [".", ",", ";"]
texto = texto.replace('\n', ' ').replace('\r', ' ')
documentos = re.split('|'.join(map(re.escape, separadores)), texto)

# # Salva o nome do arquivo em uma variável global para futuros usos
# nome_arquivo_gerado = "BagOfWords.txt"
# nome_arquivo_local = "teste.txt"

# # Lê o arquivo de texto local
# with open(nome_arquivo_local, "r") as file:
#     texto = file.read()

# # Separa o texto em documentos usando ".", ",", ";" como separadores e substitui "\n" e "\r" por um espaço em branco
# separadores = [".", ",", ";"]
# texto = texto.replace('\n', ' ').replace('\r', ' ')
# documentos = re.split('|'.join(map(re.escape, separadores)), texto)

# Remove os documentos vazios (resultantes de múltiplos caracteres separadores consecutivos)
documentos = list(filter(lambda x: len(x) > 0, documentos))

# Remove os espaços em branco do início e do final dos documentos
documentos = [documento.strip() for documento in documentos]

# Cria um set de todas as palavras encontradas nos documentos
todas_palavras = set()
for documento in documentos:
    palavras = documento.split()
    for palavra in palavras:
        todas_palavras.add(palavra)

# Cria um dicionário para contar a frequência de cada palavra em cada documento
freq_palavras = {}
for i, documento in enumerate(documentos):
    palavras = documento.split()
    for palavra in palavras:
        if palavra not in freq_palavras:
            freq_palavras[palavra] = np.zeros(len(documentos), dtype=int)
        freq_palavras[palavra][i] += 1

# Cria a matriz de frequência
matriz = np.zeros((len(documentos), len(todas_palavras)), dtype=int)
for j, palavra in enumerate(todas_palavras):
    if palavra in freq_palavras:
        matriz[:, j] = freq_palavras[palavra]

# Remove as linhas com somente zeros
linhas_nao_nulas = np.any(matriz, axis=1)
matriz = matriz[linhas_nao_nulas]
documentos = [documentos[i] for i in range(len(linhas_nao_nulas)) if linhas_nao_nulas[i]]

# Cria os rótulos de linhas e colunas
rotulos_linhas = np.arange(1, len(documentos)+1, dtype=int).reshape(-1, 1)
rotulos_colunas = np.array(list(todas_palavras))

# Concatena os rótulos de linhas e colunas e a matriz final
matriz_final = np.hstack([np.array([["Documento"]]), rotulos_colunas.reshape(1, -1)])
matriz_final = np.vstack([matriz_final, np.hstack([rotulos_linhas, matriz])])

# Salva a matriz em um arquivo de texto
np.savetxt(nome_arquivo_gerado, matriz_final, delimiter=";", fmt="%s")

print("Arquivo 'BagOfWords.txt' salvo com sucesso.")

Arquivo 'BagOfWords.txt' salvo com sucesso.


In [45]:
import numpy as np

# Função para calcular o TF-IDF
def calculate_tfidf(freq_palavra_doc, termos_doc, num_documentos, freq_termo_doc):
    if freq_termo_doc > 0 and freq_palavra_doc > 0:
        return (freq_palavra_doc / termos_doc) * np.log(num_documentos / freq_termo_doc)
    else:
        return 0

# Leitura do arquivo de entrada
with open('BagOfWords.txt', 'r') as file:
    lines = file.readlines()

# Extração dos dados do arquivo
num_documentos = len(lines) - 1
num_palavras = len(lines[0].strip().split(';')) - 1
rotulos_palavras = lines[0].strip().split(';')[1:]  # Rótulos das palavras
rotulos_documentos = [line.strip().split(';')[0] for line in lines[1:]]  # Rótulos dos documentos
matriz = np.zeros((num_documentos, num_palavras))

for i in range(1, len(lines)):
    values = lines[i].strip().split(';')[1:]
    for j in range(num_palavras):
        freq_palavra_doc = int(values[j])
        termos_doc = sum(map(int, values))
        freq_termo_doc = sum([1 for line in lines[1:] if int(line.strip().split(';')[j + 1]) > 0])
        matriz[i - 1, j] = calculate_tfidf(freq_palavra_doc, termos_doc, num_documentos, freq_termo_doc)

# Salvando a matriz de resultados em um novo arquivo
with open('TF-IDF.txt', 'w') as file:
    file.write('Documento;' + ';'.join(rotulos_palavras) + '\n')
    for i in range(num_documentos):
        file.write(rotulos_documentos[i] + ';' + ';'.join(map(str, matriz[i, :])))
        file.write('\n')

print("Arquivo 'TF-IDF.txt' salvo com sucesso.")


KeyboardInterrupt: ignored

In [41]:
import numpy as np

# Carregar a matriz do arquivo de texto
data = np.genfromtxt('TF-IDF.txt', delimiter=';', skip_header=1)
matrix = data[:, 1:]

# Calcula a similaridade para cada par de vetores
rows, cols = matrix.shape
similarity_matrix = np.zeros((rows + 1, rows + 1), dtype=object)

# Adicionar rótulos nas linhas e colunas
document_numbers = np.arange(rows)  # Números de documento
similarity_matrix[1:, 0] = document_numbers.astype(int)  # Rótulos na primeira coluna (convertidos para inteiros)
similarity_matrix[0, 1:] = document_numbers.astype(int)  # Rótulos na primeira linha (convertidos para inteiros)

for i in range(1, rows + 1):
    for j in range(1, rows + 1):
        similarity_matrix[i, j] = "{:.5f}".format(np.dot(matrix[i - 1], matrix[j - 1]) / (np.linalg.norm(matrix[i - 1]) * np.linalg.norm(matrix[j - 1])))

# Salvar a matriz de similaridade em um novo arquivo de texto
np.savetxt('CosSimilarity.txt', similarity_matrix, delimiter=';', fmt='%s')

print("Arquivo 'CosSimilarity.txt' salvo com sucesso.")